In [1]:
import numpy as np 
import pandas as pd 
import nltk 
import sklearn 
from sklearn import*
import re 
from tqdm import tqdm 
import catboost as ctb
import joblib

In [2]:
tqdm.pandas()

In [3]:
df = pd.read_csv('clean_df.csv')
df = df.dropna()

In [4]:
def hatred_upper(text):
    upper = len([i for i in text if i.isupper()])
    lower = len([i for i in text if i.islower()])
    return upper/(upper+lower)

def hatred_punc(text):
    punct = len(re.findall(r'[^\w\s]',text))
    text = len(re.findall(r'\w+',text))
    return punct/text


In [5]:
idf = feature_extraction.text.TfidfVectorizer(max_features=2000,ngram_range=(1,2))

In [6]:
idf_text=idf.fit_transform(df.clean_text).toarray()

In [7]:
df['upper']=df.ttext.apply(hatred_upper)
df['punct']=df.ttext.apply(hatred_punc)
df['title']=df.ttext.apply(lambda x: len([i for i in x if i.istitle()==True]))
df['pos_sc'] = df.ttext.apply(lambda text: len(re.findall(r'\)|D',text)))
df['neg_sc'] = df.ttext.apply(lambda text: len(re.findall(r'\(|C|c|С|c',text)))


In [8]:
df.corr()

,ttype,upper,punct,title,pos_sc,neg_sc
ttype,1.000000,0.063915,-0.003387,0.089728,0.533223,-0.347500
upper,0.063915,1.000000,0.139611,0.913370,0.076962,0.189603
punct,-0.003387,0.139611,1.000000,0.066682,0.214800,0.550125
title,0.089728,0.913370,0.066682,1.000000,0.086383,0.167231
pos_sc,0.533223,0.076962,0.214800,0.086383,1.000000,-0.180788
neg_sc,-0.347500,0.189603,0.550125,0.167231,-0.180788,1.000000


In [9]:
end_df=np.hstack([idf_text,df.iloc[:,-5:].values])

In [10]:
clf_mnb =naive_bayes.MultinomialNB(alpha=10)

In [11]:
clf_mnb.fit(end_df,df['ttype'])

MultinomialNB(alpha=10)

In [12]:
np.mean(model_selection.cross_val_score(clf_mnb,end_df,df.ttype,cv=3,scoring='roc_auc',verbose=5))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV]  ................................................................
[CV] .................................... , score=0.986, total=   3.7s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.9s remaining:    0.0s
[CV]  ................................................................
[CV] .................................... , score=0.986, total=   3.2s
[CV]  ................................................................
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    7.4s remaining:    0.0s
[CV] .................................... , score=0.988, total=   2.9s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   10.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   10.6s finished


0.986681796194624

In [13]:
def nlp_ex(text):
    # some feature engineering
    upper = [[hatred_upper(text)]]
    punctuation = [[hatred_punc(text)]]
    title = [[len([i for i in text if i.istitle()==True])]]
    pos_sc = [[len(re.findall(r'\)|D',text))]]
    neg_sc = [[len(re.findall(r'\(|C|c|С|c',text))]]
    #text preprocessing
    stemmer = nltk.stem.SnowballStemmer('russian')
    preproc=[stemmer.stem(i) for i in nltk.regexp_tokenize(text.lower(),r'[А-я]+')]
    text = ' '.join(preproc)
    idf_text= (idf.transform([text])).toarray()
    # concat type of file that we needeed
    end_text=np.hstack([idf_text,upper,punctuation,title,pos_sc,neg_sc])
    # use model
    pred = clf_mnb.predict_proba(end_text)

    return print(f" Negative {pred[0][0]:.2%}, Positive {pred[0][1]:.2%}")

In [14]:
for i in ['это хорошо','позитив',"понравился вечер"," это просто моя любовь","милота"]:
    nlp_ex(i)

Negative 37.05%, Positive 62.95%
 Negative 49.34%, Positive 50.66%
 Negative 30.67%, Positive 69.33%
 Negative 47.98%, Positive 52.02%
 Negative 49.34%, Positive 50.66%


In [15]:
for i in ['ужас','ну что за день(',"ненавижу тебя","простo тварь","это конечно фиаско..."]:
    nlp_ex(i)

Negative 82.39%, Positive 17.61%
 Negative 92.15%, Positive 7.85%
 Negative 80.94%, Positive 19.06%
 Negative 50.57%, Positive 49.43%
 Negative 47.09%, Positive 52.91%
